<a href="https://colab.research.google.com/github/Mohit03singh/AI-job-recommender/blob/main/AI_Job_Recommendator_colab_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Cell 1: Install all the required libraries
#
!pip install langchain-groq -q
!pip install langchain -q
!pip install faiss-cpu -q
!pip install langchain-community -q
!pip install PyPDF2 -q

print("All libraries have been installed!")

All libraries have been installed!


In [16]:
# Cell 2: Import libraries and Set up API Key
# Now we will import the installed libraries and set up the API key

import os
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings # We will use a local model for embeddings

# How to set the Groq API Key:
# On the left side of Colab, click on the '🔑' (Secrets) icon.
# Click 'Add a new secret'.
# Name: GROQ_API_KEY
# Value: Paste your Groq API Key here that you saved earlier.
# After this, run this cell.

try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    print("API Key has been successfully set!")
except userdata.SecretNotFoundError:
    print("ERROR: GROQ_API_KEY secret not found. Please add it in Colab Secrets.")
except Exception as e:
    print(f"An error occurred: {e}")

# IMPORTANT: We are using Ollama for embeddings, which can be a bit complex to set up in Colab.
# For simplicity, we will use a pre-computed or simpler embedding model if Ollama setup is difficult.
# Let's switch to a HuggingFace embedding model which is easier to set up in Colab.
!pip install sentence-transformers -q
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

print("Embedding model is also ready!")


API Key has been successfully set!
Embedding model is also ready!


In [17]:
# Cell 3: Preparing Job Data (The "Retrieval" part of RAG)
# In a real project, data comes from Google Jobs or LinkedIn.
# But for beginners, we will create sample data ourselves to keep the project simple.

print("Creating sample job data...")

# This is our Job Database. We have written it in text format.
jobs_data_text = """
Job Title: Python Developer Intern
Company: Tech Innovations Inc.
Location: Bangalore
Job Type: Internship
Description: Looking for a Python developer intern to work on backend systems. Must know Python, Django, and basic database concepts. Good opportunity to learn about scalable systems.

Job Title: Frontend Developer
Company: Creative Solutions
Location: Pune
Job Type: Full-time
Description: We need an experienced Frontend Developer proficient in React, HTML, CSS, and JavaScript. You will be responsible for building beautiful and responsive user interfaces. 3+ years of experience required.

Job Title: Data Science Intern
Company: Data Insights Co.
Location: Work from Home
Job Type: Internship
Description: A great opportunity for students to get hands-on experience in data analysis, machine learning, and visualization. Required skills: Python, Pandas, Matplotlib, and basic knowledge of machine learning algorithms.

Job Title: UI/UX Designer
Company: Creative Solutions
Location: Mumbai
Job Type: Full-time
Description: We are seeking a talented UI/UX designer to create amazing user experiences. The ideal candidate should have an eye for clean and artful design, possess superior UI/UX skills and be able to translate high-level requirements into interaction flows. Proficiency in Figma, Sketch, or Adobe XD is a must.

Job Title: Backend Engineer - Java
Company: FinTech Secure
Location: Bangalore
Job Type: Full-time
Description: We are hiring a Backend Engineer with strong experience in Java, Spring Boot, and Microservices. You will be building the backbone of our financial services platform. Experience with AWS and Docker is a plus.
"""

# Splitting the text into smaller documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.create_documents([jobs_data_text])

print(f"{len(documents)} documents have been created from the job data.")
# Now we will convert these documents into embeddings and save them into a Vector Store
# A Vector Store is like a smart database that helps in finding similar text
print("Creating Vector Store (FAISS)...")
vector_store = FAISS.from_documents(documents, hf_embeddings)
print("Vector Store has been successfully created!")


Creating sample job data...
2 documents have been created from the job data.
Creating Vector Store (FAISS)...
Vector Store has been successfully created!


In [18]:
# Cell 4: Creating AI Model and RAG Chain (The "Augmented Generation" part)
# Here we will use LangChain to connect the whole system

print("Preparing AI Model (Groq) and RAG chain...")

# 1. Initialize the AI Model (LLM)
llm = ChatGroq(model_name="Llama3-8b-8192")

# 2. Create Prompt Template
# This template tells the AI what it needs to do.
# We will give it the context (our jobs) and the user's question.
prompt_template = """
You are a friendly and helpful career advisor for students.
Your task is to recommend suitable jobs from the provided context based on the user's skills and preferences.
For each recommended job, provide a short, 2-3 line explanation of why it is a good match for the user.
If there are no matching jobs in the context, just say "I'm sorry, I couldn't find any suitable jobs based on your details in my current database."

Context:
{context}

User's Details:
{input}

Your Recommendations:
"""

prompt = PromptTemplate.from_template(prompt_template)

# 3. Create a chain that combines the documents (jobs) and the prompt
document_chain = create_stuff_documents_chain(llm, prompt)

# 4. Create the Retriever
# The retriever's job is to fetch relevant jobs from the Vector Store
retriever = vector_store.as_retriever()

# 5. Create the Final Retrieval Chain
# This chain will first retrieve relevant jobs based on user input,
# and then give those jobs to the LLM so it can generate recommendations.
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("RAG chain has been successfully created!")


Preparing AI Model (Groq) and RAG chain...
RAG chain has been successfully created!


In [21]:
# Cell 5: Testing the Chain (with User Input)
# Let's see if our system is working by providing input interactively.

print("--- System Test ---")
print("Please provide your details below to get job recommendations.")

# Get user input from the console
skills = input("Enter your skills (e.g., Python, data analysis, React): ")
experience = input("Enter your experience (e.g., Fresher, 3 years in Java): ")
preferences = input("Enter your preferences (e.g., Internship in Pune, work from home): ")

# Combine the user's input into a single query string.
input_query = f"Skills: {skills}. Experience: {experience}. Preferences: {preferences}."

# Call our RAG chain with the formatted input.
# Assumes 'retrieval_chain' has been created in a previous cell.
response = retrieval_chain.invoke({"input": input_query})

# Print the response from the AI.
print("\n--- AI's Response ---\n")
print(response["answer"])
print("\n--- Test Complete! ---")

--- System Test ---
Please provide your details below to get job recommendations.
Enter your skills (e.g., Python, data analysis, React): python,sql , power bi 
Enter your experience (e.g., Fresher, 3 years in Java): 1 year in data analyst
Enter your preferences (e.g., Internship in Pune, work from home): job in delhi 

--- AI's Response ---

Based on the user's skills and preferences, here are some job recommendations:

**Job Recommendation 1:** Data Science Intern (Data Insights Co.)
This internship is a great match for the user because they already have experience in data analysis and possess skills in Python, Pandas, and Matplotlib. The opportunity to work on data analysis, machine learning, and visualization will help them grow in their career.

**Job Recommendation 2:** Python Developer Intern (Tech Innovations Inc.)
This internship aligns well with the user's skills in Python, Django, and basic database concepts. The opportunity to work on backend systems and learn about scalabl

In [ ]:
# Cell 5: Chain ko Test karna (Example ke saath)
# Chaliye dekhte hain humara system kaam kar raha hai ya nahi.

print("System ko test kar rahe hain...")

# Example User Input [cite: 7]
user_input = {
    "skills": "I know Python and I have done a course on data analysis. I am a final year student.",
    "experience": "No professional experience, just college projects.",
    "preferences": "Looking for an internship, location can be anywhere, work from home is preferred."
}

# User ka input combine karke ek single query banate hain
input_query = f"Skills: {user_input['skills']}. Experience: {user_input['experience']}. Preferences: {user_input['preferences']}."

# Humari RAG chain ko call karte hain
response = retrieval_chain.invoke({"input": input_query})

# Response ko print karte hain
print("\n--- AI ka Jawab ---\n")
print(response["answer"])
print("\n--- Testing Complete! ---")

# Ek aur example
print("\n--- Dusra Example Test Kar Rahe Hain ---\n")
user_input_2 = {
    "skills": "I am a frontend developer with 3 years of experience in React and Javascript.",
    "experience": "3 years",
    "preferences": "Looking for a full-time job in Pune."
}
input_query_2 = f"Skills: {user_input_2['skills']}. Experience: {user_input_2['experience']}. Preferences: {user_input_2['preferences']}."
response_2 = retrieval_chain.invoke({"input": input_query_2})

print("\n--- AI ka Jawab (Example 2) ---\n")
print(response_2["answer"])
print("\n--- Testing Complete! ---")

System ko test kar rahe hain...

--- AI ka Jawab ---

Based on your skills and preferences, I recommend the following jobs:

1. Python Developer Intern at Tech Innovations Inc.:
This job is a great match for you since you know Python and have done a course on data analysis. As a Python developer intern, you'll have the opportunity to work on backend systems and learn about scalable systems, which aligns with your skills and interests. Plus, it's an internship, which fits your experience level and preference for a work-from-home arrangement.

2. Data Science Intern at Data Insights Co.:
This job is another good fit for you, as it requires Python skills and experience with data analysis. Since you've already done a course on data analysis, you'll have a strong foundation to apply to this role. Additionally, it's an internship, and you can work from home, which aligns with your preferences.

Based on your skills and preferences, I couldn't find any other suitable jobs in the provided cont